In [36]:
%%time

#!/usr/bin/python
# -*- coding: utf-8 -*-
# Author: Selvaria
# IEEE-CIS
# https://www.kaggle.com/xhlulu/ieee-fraud-xgboost-with-gpu-fit-in-40s

import os

import numpy as np
import pandas as pd

#这里将TransactionID设为index是为了合并时能够将二者的数据全部保存到一个表里，不会错漏
train_transaction = pd.read_csv('d:/data/Kaggle/IEEE-CIS/train_transaction.csv', index_col='TransactionID')
test_transaction = pd.read_csv('d:/data/Kaggle/IEEE-CIS/test_transaction.csv', index_col='TransactionID')

train_identity = pd.read_csv('d:/data/Kaggle/IEEE-CIS/train_identity.csv', index_col='TransactionID')
test_identity = pd.read_csv('d:/data/Kaggle/IEEE-CIS/test_identity.csv', index_col='TransactionID')

sample_submission = pd.read_csv('d:/data/Kaggle/IEEE-CIS/sample_submission.csv', index_col='TransactionID')

train = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)
test = test_transaction.merge(test_identity, how='left', left_index=True, right_index=True)

print(train.shape)
print(test.shape)

(590540, 433)
(506691, 432)
Wall time: 46.8 s


In [4]:
train.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


In [37]:
y_train = train['isFraud'].copy()
y_train.head()

TransactionID
2987000    0
2987001    0
2987002    0
2987003    0
2987004    0
Name: isFraud, dtype: int64

In [38]:
#删掉一些表，节省内存

del train_transaction, train_identity, test_transaction, test_identity

# Drop target, fill in NaNs
X_train = train.drop('isFraud', axis=1)
X_train = X_train.drop('TransactionDT', axis=1)
X_test = test.copy()
X_test = X_test.drop('TransactionDT', axis=1)


del train, test

X_train = X_train.fillna(-999) #为了模型能够识别
X_test = X_test.fillna(-999)

print(X_train.shape)
print(X_test.shape)

X_train.head()

(590540, 431)
(506691, 431)


,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,68.5,W,13926,-999.0,150.0,discover,142.0,credit,315.0,87.0,...,-999,-999.0,-999,-999,-999,-999,-999,-999,-999,-999
2987001,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,...,-999,-999.0,-999,-999,-999,-999,-999,-999,-999,-999
2987002,59.0,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,...,-999,-999.0,-999,-999,-999,-999,-999,-999,-999,-999
2987003,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,...,-999,-999.0,-999,-999,-999,-999,-999,-999,-999,-999
2987004,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


In [ ]:
# Label Encoding

from sklearn import preprocessing

for f in X_train.columns: #按列遍历，将文字数字化
    if X_train[f].dtype=='object' or X_test[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f].values) + list(X_test[f].values))
        X_train[f] = lbl.transform(list(X_train[f].values))
        X_test[f] = lbl.transform(list(X_test[f].values))
    

X_test.head()

In [27]:
#训练模型
#XGB

import xgboost as xgb

clf = xgb.XGBClassifier(
    objective='binary:logistic',
    n_estimators=500,
    max_depth=9,
    learning_rate=0.05,
    subsample=0.9, #样本采样率，即90%作为训练集
    colsample_bytree=0.9, #列（特征值）采样率，即90%的列作为特征值
    missing=-999,
    random_state=1949,
    silent = False
    #tree_method='gpu_hist'  # 开gpu用
)

In [28]:
%%time
clf.fit(X_train, y_train)

XGBoostError: b'[16:26:19] C:/Users/Administrator/Desktop/xgboost/src/objective/regression_obj.cu:66: Check failed: preds.Size() == info.labels_.Size() (506691 vs. 590540) labels are not correctly providedpreds.size=506691, label.size=590540'

In [18]:
#

test_train = np.array([[1,2,3],  
                    [1,3,4],  
                    [2,1,2],  
                    [4,5,6],  
                    [3,5,3],  
                    [1,7,2]]) 
print(test_train.shape)
test_train[:,1]

(6, 3)


array([2, 3, 1, 5, 5, 7])

In [16]:
#预测，因为比赛是要求预测概率，使用predict_proba方法
#返回的是一个 n 行 k 列的数组， 第i行 第j列上的数值是模型预测 第 i 个预测样本为j列标签的概率，并且每一行的概率和为1。

sample_submission['isFraud'] = clf.predict_proba(X_test)[:,1] #即返回第1列（标签为fraud的概率组成的列）
sample_submission.to_csv('d:/data/Kaggle/IEEE-CIS/simple_xgboost.csv')

In [19]:
sample_submission['other'] = clf.predict_proba(X_test)[:,0]
sample_submission.head()

,isFraud,other
TransactionID,,
3663549,0.003801,0.996199
3663550,0.001356,0.998644
3663551,0.003680,0.996320
3663552,0.002573,0.997427
3663553,0.001063,0.998937


In [20]:
%%time

from sklearn import linear_model

lm = linear_model.LogisticRegression()
lm.fit(X_train, y_train)

Wall time: 1min 32s


In [21]:
#逻辑回归分类

sample_submission = pd.read_csv('d:/data/Kaggle/IEEE-CIS/sample_submission.csv', index_col='TransactionID')

sample_submission['isFraud'] = lm.predict_proba(X_test)[:,1] #即返回第1列（标签为fraud的概率组成的列）
sample_submission.to_csv('d:/data/Kaggle/IEEE-CIS/logistic_regression.csv')

sample_submission.head()

,isFraud
TransactionID,
3663549,0.018773
3663550,0.023572
3663551,0.035241
3663552,0.023315
3663553,0.020394
